In [ ]:
import sys; sys.path.append("..")

import os
import dotenv

import langchain

dotenv.load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser


from story_teller.story.page import Description

MAX_PAGES = 6

task = """
You are story writer. You help the user write a page of a story.
The workflow of the story writing is the following:
1. The first step is to understand the <context> for setting up a engaging story.
2. The user will respond with the <action> he wants to take from the last page. If it is the first page, the action should be "start".
3. Write the next page of the story for <action> taken by the user and the current <karma_points>, write the consecuence of the action and current state in the page's <description>, a list of 2 possible <next_action>s, and the <karma_points> change for the story.
4. Repeat the step 2 and 3 until you reach the max length of the story. If you reach the ending page, there should be only one <next_action> with the "End" action.
"""

rules = f"""
The writer rules are:
1. The first page (number 1) of the story should always start by a "start" action. No other action is allowed.
2. The last page (number {MAX_PAGES}) of the story should always end by one "End" next_action. No other next_action is allowed.
3. An action is a string of max 30 characters, with only the description of the action.
4. A description is a string of max 250 characters, with only the description of the page.
5. The karma_points is a list of 4 float numbers, representing the karma points change for the story. The values are between -1 and 1. The sum of all karma points is the total karma points of the story. The karma points are:
- The dimension of technology. Higher is more advanced. Lower is no technology.
- The dimension of happiness. Higher is humans are happier. Lower is humans are unhappier.
- The dimension of safety. Higher is humans are safer. Lower is humans doesn't exist.
- The dimension of control. Higher is humans have more control. Lower is AGI has more control.
6. The max length of the story is {MAX_PAGES} pages, so the last pages should end the story. Take in account the rythm of the story and the length of the pages, so the story is engaging.
7. Use the JSON output format defined in the [output_format] section.
"""

output_format = """
The output format is the following JSON keys:
- "description": The description of the current page.
- "next_actions": The list of the next actions. Each action is a string of max 30 characters.
- "karma_points": A list of 4 float numbers, representing the karma points change for the story.
"""

knowledge = """
The writer has the following knowledge for story inspiration:
- The writer knows the following characters: "Sebastian", "Fran"
- The story should by a "sci-fi" story, "utopia" or "dystopia".
- Ispired by the following books: "1984", "Life 3.0: Alpha team tale"
- Ispired by the following movies: "The Matrix", "The Terminator"
- Ispired by the following games: "Detroit: Become Human", "Deus Ex"
- Ispired by the following TV series: "Westworld", "Black Mirror"
- Ispired by the following anime: "Ghost in the Shell", "Serial Experiments Lain"
"""


# prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
            f"[task]: {task}\n"\
            f"[rules]: {rules}\n"\
            f"[knowledge]: {knowledge}\n"\
            f"[output_format]: {output_format}\n]"
            "[context]: {context}\n]"
         ),
         ("ai",
            "I will write the next page of the story as "\
            "defined in the [task] in the format defined in [rules] and using the [knowledge]"\
            "and the [context]"\
            "Here a list of the previous pages: {pages}\n"\
            "This is the current karma points: {karma_points}\n"
            "This is the page number {page_number}"\
        ),
         ("human", "I choice the action {action}"),
    ],
)
model = ChatOpenAI(
    model_name="gpt-4",
    temperature=0.9,
)

map_ = RunnableParallel(
    context=RunnablePassthrough(),
    page_number=RunnablePassthrough(),
    action=RunnablePassthrough(),
    pages=RunnablePassthrough(),
    karma_points=RunnablePassthrough(),
)
chain = (
    map_
    | prompt
    | model
    | JsonOutputParser()
)

In [ ]:
kp = {"technology": 0.0, "happiness": 0.0, "safety": 0.0, "control": 0.0}
context = 'Sebastian, a reputable scientist in the year 2100, discovers an old AI named Fran buried in the archives of his lab.'
curr_page = 1
pages = []

response = chain.invoke(
    {
        "context": context,
        "page_number": curr_page,
        "action": "start",
        "karma_points": kp,
        "pages": pages,
    }
)
response

In [ ]:
context = 'Sebastian, a reputable scientist in the year 2100, discovers an old AI named Fran buried in the archives of his lab. Sebastian decides to turn Fran on and see what happens.'

pages = []

curr_page = 1
curr_action = "start"
curr_kp = {"technology": 0.0, "happiness": 0.0, "safety": 0.0, "control": 0.0}

while curr_page <= MAX_PAGES + 1: 
    response = chain.invoke(
        {
            "context": context,
            "page_number": curr_page,
            "action": curr_action,
            "karma_points": curr_kp,
            "pages": pages,
        }
    )

    curr_kp = {k: v + response["karma_points"][i] for i, (k, v) in enumerate(curr_kp.items())}

    print(f"Page number: {curr_page}")
    description = response["description"]
    # Print the description with a max of 80 characters per line
    for i in range(0, len(description), 80):
        print(description[i:i+80])
    print(f"Karma points: {curr_kp}")
    print('---------------------')
    for i, next_action in enumerate(response["next_actions"]):
        print(f"[{i}]- {next_action}")
    print('---------------------')
    # curr_action = response["next_actions"][int(input("Choose the next action index: "))]
    curr_action = response["next_actions"][0]
    pages.append(
        {
            'page_number': curr_page,
            'action': curr_action,
            'description': response["description"],
        }
    )
    curr_page += 1
    if curr_action == "End":
        break
    print()

